In [1]:
import pandas as pd
import numpy as np

# Import the Data

In [2]:
df_xwalk = pd.read_csv('data/ca_xwalk.csv', low_memory=False)
df_all_ee = pd.read_csv('data/ca_od_main_JT00_2018.csv', low_memory=False)
df_primary_ee = pd.read_csv('data/ca_od_main_JT01_2018.csv', low_memory=False)

# Merge geo data into the employment data

In [3]:
# I'm not worrying about memory, so we create two copies of df_xwalk with different column names
df_xwalk_work = df_xwalk.copy(deep=True)
df_xwalk_work.columns = 'w_' + df_xwalk_work.columns.values
df_xwalk_res = df_xwalk.copy(deep=True)
df_xwalk_res.columns = 'h_' + df_xwalk.columns.values

# Now we do two merges on both employment dataframes
# It's single threaded, so it takes awhile
df_all = df_all_ee.merge(
    right=df_xwalk_work,
    left_on='w_geocode',
    right_on='w_tabblk2010'
).merge(
    right=df_xwalk_res,
    left_on='h_geocode',
    right_on='h_tabblk2010'
)
df_primary = df_primary_ee.merge(
    right=df_xwalk_work,
    left_on='w_geocode',
    right_on='w_tabblk2010'
).merge(
    right=df_xwalk_res,
    left_on='h_geocode',
    right_on='h_tabblk2010'
)


# Create place dataframes for comparison

In [7]:
# We create dataframes for people working in an official census place and those
# employed people living in each official census place 

df_w_primary_place = df_primary.groupby(
    ['w_stplcname']).agg(
        {'w_cbsaname': [pd.Series.mode], 'w_ctyname': [pd.Series.mode], 'S000': ['sum']}
)
df_h_primary_place = df_primary.groupby(
    ['h_stplcname']).agg(
        {'S000': ['sum']}
)
df_w_all_place = df_all.groupby(
    ['w_stplcname']).agg(
        {'w_cbsaname': [pd.Series.mode], 'w_ctyname': [pd.Series.mode],'S000': ['sum']}
)
df_h_all_place = df_all.groupby(
    ['h_stplcname']).agg(
        {'S000': ['sum']}
)
lst_w_col_names = ['CBSA', 'County', 'jobs_in_loc']
lst_h_col_names = ['employed_residents']
df_w_primary_place.columns = lst_w_col_names
df_w_all_place.columns = lst_w_col_names
df_h_primary_place.columns = lst_h_col_names
df_h_all_place.columns = lst_h_col_names

# Now we combine the dataframes
df_primary_place = df_w_primary_place.join(df_h_primary_place['employed_residents'])
df_primary_place.index.rename('Census Place Name', inplace=True)
df_all_place = df_w_all_place.join(df_h_all_place['employed_residents'])
df_all_place.index.rename('Census Place Name', inplace=True)

# Next we add calculated columns
df_primary_place['worker_deficit'] = df_primary_place['jobs_in_loc'] - df_primary_place['employed_residents']
df_primary_place['jobs_emp_resident_ratio'] = df_primary_place['jobs_in_loc'] / df_primary_place['employed_residents']
df_all_place['worker_deficit'] = df_all_place['jobs_in_loc'] - df_primary_place['employed_residents']
df_all_place['jobs_emp_resident_ratio'] = df_all_place['jobs_in_loc'] / df_primary_place['employed_residents']

# Some top lists

In [8]:
print('Top 50 Worst Jobs to Employed Resident Ratios: All California')
print('At least 10,000 primary jobs in location')
df_primary_place[df_primary_place['jobs_in_loc'] > 10000].sort_values(['jobs_emp_resident_ratio'], ascending=False).head(50)

Top 50 Worst Jobs to Employed Resident Ratios: All California
At least 10,000 primary jobs in location


,CBSA,County,jobs_in_loc,employed_residents,worker_deficit,jobs_emp_resident_ratio
Census Place Name,,,,,,
"Industry city, CA","Los Angeles-Long Beach-Anaheim, CA","Los Angeles County, CA",65393,123.0,65270.0,531.650407
"Vernon city, CA","Los Angeles-Long Beach-Anaheim, CA","Los Angeles County, CA",35823,306.0,35517.0,117.068627
"Irwindale city, CA","Los Angeles-Long Beach-Anaheim, CA","Los Angeles County, CA",16613,714.0,15899.0,23.267507
"Commerce city, CA","Los Angeles-Long Beach-Anaheim, CA","Los Angeles County, CA",47850,4826.0,43024.0,9.915044
"El Segundo city, CA","Los Angeles-Long Beach-Anaheim, CA","Los Angeles County, CA",67765,8254.0,59511.0,8.209959
"Santa Fe Springs city, CA","Los Angeles-Long Beach-Anaheim, CA","Los Angeles County, CA",55512,7721.0,47791.0,7.189742
"Stanford CDP, CA","San Jose-Sunnyvale-Santa Clara, CA","Santa Clara County, CA",11596,3094.0,8502.0,3.747899
"Westlake Village city, CA","Los Angeles-Long Beach-Anaheim, CA","Los Angeles County, CA",12970,3495.0,9475.0,3.711016
"Emeryville city, CA","San Francisco-Oakland-Berkeley, CA","Alameda County, CA",22679,6315.0,16364.0,3.591291


In [9]:
print('Top 20 Worst Jobs to Employed Resident Rations: Bay Area')
print('At least 1000 primary jobs in location')
df_bay_area = df_primary_place.query(
    "CBSA in ['San Jose-Sunnyvale-Santa Clara, CA', 'San Francisco-Oakland-Berkeley, CA']")\
    .query(
    "jobs_in_loc > 1000"
    )
df_bay_area.sort_values(['jobs_emp_resident_ratio'], ascending=False).head(25)

Top 20 Worst Jobs to Employed Resident Rations: Bay Area
At least 1000 primary jobs in location


,CBSA,County,jobs_in_loc,employed_residents,worker_deficit,jobs_emp_resident_ratio
Census Place Name,,,,,,
"Colma town, CA","San Francisco-Oakland-Berkeley, CA","San Mateo County, CA",4238,1033.0,3205.0,4.102614
"Stanford CDP, CA","San Jose-Sunnyvale-Santa Clara, CA","Santa Clara County, CA",11596,3094.0,8502.0,3.747899
"Emeryville city, CA","San Francisco-Oakland-Berkeley, CA","Alameda County, CA",22679,6315.0,16364.0,3.591291
"Palo Alto city, CA","San Jose-Sunnyvale-Santa Clara, CA","Santa Clara County, CA",103582,28846.0,74736.0,3.590862
"Menlo Park city, CA","San Francisco-Oakland-Berkeley, CA","San Mateo County, CA",46559,15004.0,31555.0,3.103106
"Contra Costa Centre CDP, CA","San Francisco-Oakland-Berkeley, CA","Contra Costa County, CA",6715,2406.0,4309.0,2.790939
"Brisbane city, CA","San Francisco-Oakland-Berkeley, CA","San Mateo County, CA",6203,2383.0,3820.0,2.603021
"Burlingame city, CA","San Francisco-Oakland-Berkeley, CA","San Mateo County, CA",30342,13212.0,17130.0,2.296549
"Mountain View city, CA","San Jose-Sunnyvale-Santa Clara, CA","Santa Clara County, CA",87395,41174.0,46221.0,2.122577
